# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("city_weather_data.csv")

city_data.head()


,Unnamed: 0,City,Lat,Lon,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
0,0,São Félix do Xingu,-6.64,-51.99,88.27,48,28,4.16,BR,1593285655
1,1,Airai,-8.93,125.41,55.80,100,86,3.15,TL,1593285629
2,2,Tiznit Province,29.58,-9.50,71.40,59,0,3.49,MA,1593286215
3,3,Cape Town,-33.93,18.42,61.00,71,75,6.93,ZA,1593285481
4,4,Hithadhoo,-0.60,73.08,84.83,73,44,8.19,MV,1593285642


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:

locations = city_data[["Lat", "Lon"]].astype(float)
humidity = city_data["Humidity (%)"].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrowing database to max_temp between 70 and 80, Wind Speed <10, and zero cloudiness

ideal_weather = city_data.loc[(city_data["Max Temp (F)"] > 70) &
                             (city_data["Max Temp (F)"] < 80) &
                             (city_data["Wind Speed (MPH)"] < 10) &
                             (city_data["Cloudiness (%)"] == 0)]
                              
                              # just in case added condition to drop choices to 10
 #                            (city_data["Humidity (%)"] <=80)]
#Choosing wanted columns 
hotel_df = ideal_weather[["City", "Lat", "Lon", "Max Temp (F)", "Humidity (%)", 
                          "Cloudiness (%)", "Wind Speed (MPH)", "Country", "Date"]]
hotel_df

,City,Lat,Lon,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
2,Tiznit Province,29.58,-9.50,71.40,59,0,3.49,MA,1593286215
12,Mahajanga,-15.72,46.32,71.60,88,0,2.24,MG,1593286216
35,Zabul Province,32.25,67.25,73.04,18,0,8.12,AF,1593286218
66,Kiryat Gat,31.61,34.76,79.00,87,0,4.43,IL,1593285627
133,Nikolskoye,59.70,30.79,71.60,49,0,4.47,RU,1593286232
175,Boueni,-12.90,45.08,75.20,83,0,8.05,YT,1593286237
202,Erzin,36.96,36.20,77.00,89,0,0.54,TR,1593286241
213,Mitsamiouli,-11.38,43.28,77.74,78,0,5.48,KM,1593286242
304,Nizhniy Baskunchak,48.22,46.83,72.48,36,0,6.44,RU,1593286253
320,Wete,-5.06,39.73,74.77,86,0,8.28,TZ,1593286255


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lon,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date,Hotel Name
2,Tiznit Province,29.58,-9.50,71.40,59,0,3.49,MA,1593286215,
12,Mahajanga,-15.72,46.32,71.60,88,0,2.24,MG,1593286216,
35,Zabul Province,32.25,67.25,73.04,18,0,8.12,AF,1593286218,
66,Kiryat Gat,31.61,34.76,79.00,87,0,4.43,IL,1593285627,
133,Nikolskoye,59.70,30.79,71.60,49,0,4.47,RU,1593286232,
175,Boueni,-12.90,45.08,75.20,83,0,8.05,YT,1593286237,
202,Erzin,36.96,36.20,77.00,89,0,0.54,TR,1593286241,
213,Mitsamiouli,-11.38,43.28,77.74,78,0,5.48,KM,1593286242,
304,Nizhniy Baskunchak,48.22,46.83,72.48,36,0,6.44,RU,1593286253,
320,Wete,-5.06,39.73,74.77,86,0,8.28,TZ,1593286255,


In [7]:
# geocoordinates


target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
         "radius": target_radius,
         "types": target_type,
         "key": g_key,
          }

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lon = row["Lon"]
    
    # add keyword to params dict
    params['location'] = f"{lat},{lon}"

    # assemble url and make API request
    
    
    response = requests.get(base_url, params=params).json()
    
    try:
               
        hotel_df.loc[index, 'Hotel Name'] = response["results"][0]['name']
        
    except :
        
        print("Missing field/result... skipping.")
        
        

Missing field/result... skipping.
Missing field/result... skipping.


In [8]:
response

{'html_attributions': [],
 'next_page_token': 'CqQCHAEAACNwHa87RH8ApUZTpE5ITGDZO3cl5_vSpuCwkUTgeyxx4t5w1QPRDs4wcNIkFGOgDJKEKnjsFZ-j8JJ9d7HNqABIIP5pDlhXaOur7JNYaCXLZizbsYi3Ajmnpl4dnsdJ4_T0vN9h2Jb8m8LdPNzdYTWMeOEbQibevwfubYn43GqiGIvv_qDcu7R958RLLTOvnHcNTH6vHzWrzQRwzVMh2Snee-chrulUp_8gs8bA8N2Fz0YEg7ntW6xBDZ_9IoiFslmkrOOARQeaXLjhiYFcedmuU7K-irm5qvEdBVsNXg7B8fx96xnI1_eqvKONzGPfDYaq8QL821JkVXC0nVqaQo4xW1qvxmETxjtp1TnrJhajDUvknWMaJ-qOsL154gjCTxIQoE9gyFJKj9ulrtrQJ2vz2xoUuP5hB0cbDDvP9xu6SewZPDfBxec',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 31.5211719, 'lng': 34.4472038},
    'viewport': {'northeast': {'lat': 31.5225545802915,
      'lng': 34.4485136802915},
     'southwest': {'lat': 31.5198566197085, 'lng': 34.4458157197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'id': '2dc930633eac6827b870f8883146ff27448e1834',
   'name': 'رابطة الفنانين الفلسطينيين',
   'place_id': 'ChIJWwIK1hZ__RQRVhskkD4mrDc',
   'plus

In [10]:
hotel_df

,City,Lat,Lon,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date,Hotel Name
2,Tiznit Province,29.58,-9.50,71.40,59,0,3.49,MA,1593286215,
12,Mahajanga,-15.72,46.32,71.60,88,0,2.24,MG,1593286216,Restaurant Greedy Coco Lodge Majunga
35,Zabul Province,32.25,67.25,73.04,18,0,8.12,AF,1593286218,
66,Kiryat Gat,31.61,34.76,79.00,87,0,4.43,IL,1593285627,רגע בנחת
133,Nikolskoye,59.70,30.79,71.60,49,0,4.47,RU,1593286232,Tourist House - Sablino
175,Boueni,-12.90,45.08,75.20,83,0,8.05,YT,1593286237,Auberge bouloulou( inter djasmire )
202,Erzin,36.96,36.20,77.00,89,0,0.54,TR,1593286241,Artemis Otel
213,Mitsamiouli,-11.38,43.28,77.74,78,0,5.48,KM,1593286242,Foyer ADM
304,Nizhniy Baskunchak,48.22,46.83,72.48,36,0,6.44,RU,1593286253,Arman hotel
320,Wete,-5.06,39.73,74.77,86,0,8.28,TZ,1593286255,Sharook Riviera Grand Lodge


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [12]:
# Add marker layer ontop of heat map
hotel_list = hotel_df["Hotel Name"]
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(150, 0, 0, 0.4)',
    stroke_color='rgba(0, 150, 0, 0.4)', scale=2,
    info_box_content=[f"Hotel: {hotel}" for hotel in hotel_list]
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)



# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))